# Import libraries

In [1]:
import os
import sys
import pypots
import numpy as np
import benchpots
import matplotlib.pyplot as plt
from pypots.optim import Adam
from pypots.imputation import SAITS, BRITS, USGAN, GPVAE, MRNN
from pypots.utils.random import set_random_seed
from functions.toolkits import toolkits
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from MAEModify.error import calc_mae


2025-05-10 00:54:48.163005: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746849288.180750  687381 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746849288.186441  687381 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-10 00:54:48.204449: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidge


████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 



# Load dataset

In [2]:
set_random_seed()
physionet2012_dataset = benchpots.datasets.preprocess_physionet2012(subset="all", rate=0.1)
print(physionet2012_dataset.keys())

2025-05-10 00:54:51 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-05-10 00:54:51 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-05-10 00:54:51 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-05-10 00:54:51 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-05-10 00:54:51 [INFO]: Loaded successfully!
2025-05-10 00:55:06 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-05-10 00:55:06 [INFO]: 68807 values masked out in the val set as ground truth, take 9.97% of the original observed values
2025-05-10 00:55:06 [INFO]: 86319 values masked out in the test set as ground truth, take 9.99% of the original observed v

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'val_X_ori', 'test_X_ori'])


In [3]:
dataset_for_training = {
    "X": physionet2012_dataset['train_X'],
}

dataset_for_validating = {
    "X": physionet2012_dataset['val_X'],
    "X_ori": physionet2012_dataset['val_X_ori'],
}

dataset_for_testing = {
    "X": physionet2012_dataset['test_X'],
}

test_X_indicating_mask = np.isnan(physionet2012_dataset['test_X_ori']) ^ np.isnan(physionet2012_dataset['test_X'])
test_X_ori = np.nan_to_num(physionet2012_dataset['test_X_ori']) 

# Train/Load Models

## SAITS

In [4]:
saits = SAITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  
    MIT_weight=1,
    batch_size=32,
    epochs=10,
    patience=3,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    model_saving_strategy="best",
)

2025-05-10 00:55:06 [INFO]: No given device, using default device: cpu
2025-05-10 00:55:06 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-10 00:55:06 [INFO]: Using customized MAE as the training loss function.
2025-05-10 00:55:06 [INFO]: Using customized MSE as the validation metric function.
2025-05-10 00:55:06 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


In [5]:
saits.load("../mae/tutorial_results/imputation/saits/20250422_T181642/SAITS.pypots")

2025-05-10 00:55:06 [INFO]: Model loaded successfully from ../mae/tutorial_results/imputation/saits/20250422_T181642/SAITS.pypots


## BRITS

In [6]:
brits = BRITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    epochs=10,
    patience=3,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    model_saving_strategy="best",
)

2025-05-10 00:55:06 [INFO]: No given device, using default device: cpu
2025-05-10 00:55:06 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-10 00:55:06 [INFO]: Using customized MAE as the training loss function.
2025-05-10 00:55:06 [INFO]: Using customized MSE as the validation metric function.
2025-05-10 00:55:06 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


In [7]:
brits.load("../mae/tutorial_results/imputation/brits/20250422_T181643/BRITS.pypots")

2025-05-10 00:55:06 [INFO]: Model loaded successfully from ../mae/tutorial_results/imputation/brits/20250422_T181643/BRITS.pypots


## US-GAN

In [8]:
us_gan = USGAN(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    epochs=10,
    patience=3,
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    model_saving_strategy="best",
)

2025-05-10 00:55:06 [INFO]: No given device, using default device: cpu
2025-05-10 00:55:06 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-10 00:55:06 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


In [9]:
us_gan.load("../mae/tutorial_results/imputation/us_gan/20250422_T181643/USGAN.pypots")

2025-05-10 00:55:06 [INFO]: Model loaded successfully from ../mae/tutorial_results/imputation/us_gan/20250422_T181643/USGAN.pypots


## GP-VAE

In [10]:
gp_vae = GPVAE(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    epochs=10,
    patience=3,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    model_saving_strategy="best",
)

2025-05-10 00:55:06 [INFO]: No given device, using default device: cpu
2025-05-10 00:55:06 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-10 00:55:06 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


In [11]:
gp_vae.load("../mae/tutorial_results/imputation/gp_vae/20250422_T181643/GPVAE.pypots")

2025-05-10 00:55:06 [INFO]: Model loaded successfully from ../mae/tutorial_results/imputation/gp_vae/20250422_T181643/GPVAE.pypots


## MRNN

In [12]:
mrnn = MRNN(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,
    epochs=10,
    patience=3,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    model_saving_strategy="best",
)

2025-05-10 00:55:06 [INFO]: No given device, using default device: cpu
2025-05-10 00:55:06 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-10 00:55:06 [INFO]: Using customized RMSE as the training loss function.
2025-05-10 00:55:06 [INFO]: Using customized MSE as the validation metric function.
2025-05-10 00:55:06 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


In [13]:
mrnn.load("../mae/tutorial_results/imputation/mrnn/20250422_T181643/MRNN.pypots")

2025-05-10 00:55:06 [INFO]: Model loaded successfully from ../mae/tutorial_results/imputation/mrnn/20250422_T181643/MRNN.pypots


# Imputation Models

## SAITS

In [14]:
saits_results = saits.predict(dataset_for_testing)
saits_imputation = saits_results["imputation"]

## BRITS

In [15]:
brits_results = brits.predict(dataset_for_testing)
brits_imputation = brits_results["imputation"]

## US-GAN

In [16]:
us_gan_results = us_gan.predict(dataset_for_testing)
us_gan_imputation = us_gan_results["imputation"]

## GP-VAE

In [17]:
gp_vae_results = gp_vae.predict(dataset_for_testing)
gp_vae_imputation = gp_vae_results["imputation"]

## MRNN

In [18]:
mrnn_results = mrnn.predict(dataset_for_testing)
mrnn_imputation = mrnn_results["imputation"]

# AE/MAE Models

## SAITS

In [19]:
saits_mae, saits_ae = calc_mae(
    saits_imputation,
    test_X_ori,
    test_X_indicating_mask,
)

## BRITS

In [20]:
brits_mae, brits_ae = calc_mae(
    brits_imputation,
    test_X_ori,
    test_X_indicating_mask,
)

## US-GAN

In [21]:
usgan_mae, usgan_ae = calc_mae(
    us_gan_imputation,
    test_X_ori,
    test_X_indicating_mask,
)

## GP-VAE

In [22]:
gp_vae_imputation = np.squeeze(gp_vae_imputation, axis=1)

gpvae_mae, gpvae_ae = calc_mae(
    gp_vae_imputation,
    test_X_ori,
    test_X_indicating_mask,
)

## MRNN

In [23]:
mrnn_mae, mrnn_ae = calc_mae(
    mrnn_imputation,
    test_X_ori,
    test_X_indicating_mask,
)

# Gini Models

In [24]:
test_X_indicating_mask = test_X_indicating_mask.reshape(len(test_X_indicating_mask) * 48,  37)

## SAITS

In [25]:
saits_ae = saits_ae.reshape(len(saits_ae) * 48 , 37)

In [26]:
saits_ae = toolkits.reshape_variables_v2(saits_ae)

In [27]:
test_X_indicating_mask = toolkits.reshape_variables_v2(test_X_indicating_mask)

In [28]:
saits_ae_mask = []
aux_ae = []

for i in range(len(test_X_indicating_mask)):
    for j in range(len(test_X_indicating_mask[i])):
        if(test_X_indicating_mask[i][j] == True):
            aux_ae.append(saits_ae[i][j])
    
    saits_ae_mask.append(np.array(aux_ae))
    aux_ae = []


In [29]:
saits_gini = []

for i in range(len(saits_ae_mask)):
    saits_gini.append(toolkits.gini(saits_ae_mask[i]))

In [30]:
saits_gini

[np.float64(0.6755091355898706),
 np.float64(0.7506986500270829),
 np.float64(0.8066109439055715),
 np.float64(0.4087511422530157),
 np.float64(0.49229335494951165),
 np.float64(0.6521102841529713),
 np.float64(0.3471228353452396),
 np.float64(0.5824099403794447),
 np.float64(0.5443742162685035),
 np.float64(0.5719282990441206),
 np.float64(0.6232680118979477),
 np.float64(0.5254114700649812),
 np.float64(0.4518452356129381),
 np.float64(0.4485866742540714),
 np.float64(0.49259452766233447),
 np.float64(0.47265986390708625),
 np.float64(0.5217582646944443),
 np.float64(0.5653550172684987),
 np.float64(0.4704282416775123),
 np.float64(0.4866896131190297),
 np.float64(0.6010335376692917),
 np.float64(0.5749235694844477),
 np.float64(0.6001113584961808),
 np.float64(0.4522360878871514),
 np.float64(0.4802841788953083),
 np.float64(0.5377589677071799),
 np.float64(0.49649261170346093),
 np.float64(0.4479533545436629),
 np.float64(0.5723561830460957),
 np.float64(0.520474125299802),
 np.flo

## BRITS

In [31]:
brits_ae = brits_ae.reshape(len(brits_ae) * 48 , 37)

In [32]:
brits_ae = toolkits.reshape_variables_v2(brits_ae)

In [33]:
brits_ae_mask = []
aux_ae = []

for i in range(len(test_X_indicating_mask)):
    for j in range(len(test_X_indicating_mask[i])):
        if(test_X_indicating_mask[i][j] == True):
            aux_ae.append(brits_ae[i][j])
    
    brits_ae_mask.append(np.array(aux_ae))
    aux_ae = []


In [34]:
brits_gini = []

for i in range(len(brits_ae_mask)):
    brits_gini.append(toolkits.gini(brits_ae_mask[i]))

In [35]:
brits_gini

[np.float64(0.6175804858262515),
 np.float64(0.6762765385139351),
 np.float64(0.828616599357066),
 np.float64(0.3945630738280437),
 np.float64(0.5563239882048077),
 np.float64(0.6681562846210098),
 np.float64(0.35947764304177365),
 np.float64(0.6697165452673239),
 np.float64(0.5909322386604194),
 np.float64(0.6153092463006595),
 np.float64(0.6313128427624324),
 np.float64(0.5164445220216429),
 np.float64(0.43177765415626546),
 np.float64(0.4365326045583773),
 np.float64(0.5075631063056985),
 np.float64(0.4788337995385683),
 np.float64(0.5448889409093929),
 np.float64(0.6298976024704059),
 np.float64(0.4227358297363353),
 np.float64(0.48237123154762385),
 np.float64(0.5331634121605795),
 np.float64(0.6579205322508939),
 np.float64(0.500572190205383),
 np.float64(0.4685271366098356),
 np.float64(0.6541446757486307),
 np.float64(0.5338694921684837),
 np.float64(0.44986955671661355),
 np.float64(0.4489085369805983),
 np.float64(0.5653098133983734),
 np.float64(0.5583426436940804),
 np.floa

## US-GAN

In [36]:
usgan_ae = usgan_ae.reshape(len(usgan_ae) * 48 , 37)

In [37]:
usgan_ae = toolkits.reshape_variables_v2(usgan_ae)

In [38]:
usgan_ae_mask = []
aux_ae = []

for i in range(len(test_X_indicating_mask)):
    for j in range(len(test_X_indicating_mask[i])):
        if(test_X_indicating_mask[i][j] == True):
            aux_ae.append(usgan_ae[i][j])
    
    usgan_ae_mask.append(np.array(aux_ae))
    aux_ae = []


In [39]:
usgan_gini = []

for i in range(len(usgan_ae_mask)):
    usgan_gini.append(toolkits.gini(usgan_ae_mask[i]))

In [40]:
usgan_gini

[np.float64(0.6322594201687439),
 np.float64(0.6212527322809012),
 np.float64(0.5964329083302113),
 np.float64(0.39165814014934197),
 np.float64(0.47911384836298065),
 np.float64(0.5089992382473347),
 np.float64(0.35330622719317284),
 np.float64(0.5447303567674084),
 np.float64(0.5333393202615901),
 np.float64(0.5342287498390583),
 np.float64(0.5555584391032145),
 np.float64(0.45506277847391363),
 np.float64(0.45078963248424286),
 np.float64(0.45040227661612775),
 np.float64(0.4976606283382681),
 np.float64(0.465640885470205),
 np.float64(0.4342420230974977),
 np.float64(0.57594404635367),
 np.float64(0.4568751604292023),
 np.float64(0.4843524313703367),
 np.float64(0.5320123020828089),
 np.float64(0.6119585988167473),
 np.float64(0.5041825359722252),
 np.float64(0.44033179132071987),
 np.float64(0.5654678077913236),
 np.float64(0.49350860078897596),
 np.float64(0.45650674083247056),
 np.float64(0.4447451902371564),
 np.float64(0.5299661426126279),
 np.float64(0.5152352936190872),
 np.

## GPVAE

In [41]:
gpvae_ae = gpvae_ae.reshape(len(gpvae_ae) * 48 , 37)

In [42]:
gpvae_ae = toolkits.reshape_variables_v2(gpvae_ae)

In [43]:
gpvae_ae_mask = []
aux_ae = []

for i in range(len(test_X_indicating_mask)):
    for j in range(len(test_X_indicating_mask[i])):
        if(test_X_indicating_mask[i][j] == True):
            aux_ae.append(gpvae_ae[i][j])
    
    gpvae_ae_mask.append(np.array(aux_ae))
    aux_ae = []


In [44]:
gpvae_gini = []

for i in range(len(gpvae_ae_mask)):
    gpvae_gini.append(toolkits.gini(gpvae_ae_mask[i]))

In [45]:
gpvae_gini

[np.float64(0.529690336122872),
 np.float64(0.5942045419239153),
 np.float64(0.6796528708576146),
 np.float64(0.3689622575888245),
 np.float64(0.4499780296310951),
 np.float64(0.532503045317795),
 np.float64(0.34973165867899625),
 np.float64(0.5672355980536876),
 np.float64(0.4740431487797334),
 np.float64(0.468665006136737),
 np.float64(0.4675383228183727),
 np.float64(0.4750446335019789),
 np.float64(0.44846264044533424),
 np.float64(0.43198613415282594),
 np.float64(0.46282662645384987),
 np.float64(0.4658137696747624),
 np.float64(0.478466504689222),
 np.float64(0.47150396302285436),
 np.float64(0.5880221576240088),
 np.float64(0.4752665197503685),
 np.float64(0.4542321683183759),
 np.float64(0.4572826927291753),
 np.float64(0.44046958225042987),
 np.float64(0.4609732533708151),
 np.float64(0.47517812173376783),
 np.float64(0.47458667627499507),
 np.float64(0.42429527722273197),
 np.float64(0.4431502094248523),
 np.float64(0.4882924519324454),
 np.float64(0.45663377760113044),
 np.

## MRNN

In [46]:
mrnn_ae = mrnn_ae.reshape(len(mrnn_ae) * 48 , 37)

In [47]:
mrnn_ae = toolkits.reshape_variables_v2(mrnn_ae)

In [48]:
mrnn_ae_mask = []
aux_ae = []

for i in range(len(test_X_indicating_mask)):
    for j in range(len(test_X_indicating_mask[i])):
        if(test_X_indicating_mask[i][j] == True):
            aux_ae.append(mrnn_ae[i][j])
    
    mrnn_ae_mask.append(np.array(aux_ae))
    aux_ae = []


In [49]:
mrnn_gini = []

for i in range(len(mrnn_ae_mask)):
    mrnn_gini.append(toolkits.gini(mrnn_ae_mask[i]))

In [50]:
mrnn_gini

[np.float64(0.4572991529198507),
 np.float64(0.32569178883591476),
 np.float64(0.41114742811749183),
 np.float64(0.3844172692450798),
 np.float64(0.35688990332182824),
 np.float64(0.38802781644452256),
 np.float64(0.360935140902759),
 np.float64(0.39775125022725666),
 np.float64(0.42480380229985326),
 np.float64(0.37671776105164256),
 np.float64(0.30507775503526724),
 np.float64(0.4354567881975413),
 np.float64(0.4540973452702033),
 np.float64(0.4180823471988213),
 np.float64(0.40865616946713934),
 np.float64(0.4495262367815256),
 np.float64(0.3578498488526711),
 np.float64(0.43276006965677327),
 np.float64(0.14077117855421273),
 np.float64(0.4624022214565817),
 np.float64(0.42066126727973296),
 np.float64(0.42953930836482446),
 np.float64(0.40497087039544266),
 np.float64(0.45815108554924655),
 np.float64(0.45954139163772423),
 np.float64(0.414010116110207),
 np.float64(0.40866008476854687),
 np.float64(0.42403045991896904),
 np.float64(0.5236073264230532),
 np.float64(0.4122476156428

# IC Models

## SAITS

In [51]:
bootstrap_saits = []

for i in range(len(saits_ae_mask)):
    bootstrap_saits.append(toolkits.bootstrap_v2(saits_ae_mask, i, 9000))

In [52]:
lower_bound_saits = []
upper_bound_saits = []

for i in range(len(bootstrap_saits)):
    lower_bound_saits.append(np.percentile(bootstrap_saits[i], 2.5))
    upper_bound_saits.append(np.percentile(bootstrap_saits[i], 97.5))

print(lower_bound_saits)
print(upper_bound_saits)

[np.float64(0.5435100202757382), np.float64(0.6389517033853522), np.float64(0.6946863077170858), np.float64(0.3634579311834109), np.float64(0.47079322266190776), np.float64(0.5788795671171321), np.float64(0.2505328369706727), np.float64(0.5359367039997764), np.float64(0.5314053904702729), np.float64(0.5582454200626127), np.float64(0.6129390206193398), np.float64(0.49689047374883505), np.float64(0.4312899522539368), np.float64(0.43159689969416143), np.float64(0.48572429860568356), np.float64(0.449879075193115), np.float64(0.4872420601685669), np.float64(0.5402514549323488), np.float64(0.4547727387240157), np.float64(0.44747636759860077), np.float64(0.588078877124718), np.float64(0.5607873943935131), np.float64(0.5836259092018314), np.float64(0.42977249795158656), np.float64(0.4609105961379078), np.float64(0.5194343350930829), np.float64(0.4716154640173531), np.float64(0.4372355081746939), np.float64(0.4983646901925138), np.float64(0.5092987740476734), np.float64(0.5021311887394649), np.

In [53]:
mean_bound_saits = []

for i in range(len(lower_bound_saits)):
    mean_bound_saits.append((lower_bound_saits[i]+upper_bound_saits[i])/2)

print(mean_bound_saits)

[np.float64(0.6512606715207199), np.float64(0.7193784048414222), np.float64(0.7719023631164863), np.float64(0.40689248148923646), np.float64(0.4918988205623728), np.float64(0.6377318564462264), np.float64(0.3412823407129682), np.float64(0.5798599816719032), np.float64(0.5440548328533261), np.float64(0.5713696121807519), np.float64(0.6229507100641942), np.float64(0.5241553515562097), np.float64(0.4513667510066057), np.float64(0.44884052116810313), np.float64(0.492518669155238), np.float64(0.47209716820272307), np.float64(0.5197111647221075), np.float64(0.565393692779862), np.float64(0.47004153353056277), np.float64(0.4886920238531335), np.float64(0.6009966803246607), np.float64(0.5748301263847131), np.float64(0.6003694994012074), np.float64(0.45195908322083234), np.float64(0.48041272361026766), np.float64(0.5371019351699838), np.float64(0.4959125357742892), np.float64(0.4481209980725829), np.float64(0.5702833013297741), np.float64(0.5204102928290957), np.float64(0.5247078656146278), np.

## BRITS

In [54]:
bootstrap_brits = []

for i in range(len(brits_ae_mask)):
    bootstrap_brits.append(toolkits.bootstrap_v2(brits_ae_mask, i, 9000))

In [55]:
lower_bound_brits = []
upper_bound_brits = []

for i in range(len(bootstrap_brits)):
    lower_bound_brits.append(np.percentile(bootstrap_brits[i], 2.5))
    upper_bound_brits.append(np.percentile(bootstrap_brits[i], 97.5))

print(lower_bound_brits)
print(upper_bound_brits)

[np.float64(0.478584790231987), np.float64(0.5324684841311105), np.float64(0.7424264763855235), np.float64(0.3509549342994489), np.float64(0.5317157529916134), np.float64(0.5892287810978972), np.float64(0.2596704080558105), np.float64(0.6388700257599292), np.float64(0.572688991735357), np.float64(0.6023625025240293), np.float64(0.6217054626170403), np.float64(0.4871980988058966), np.float64(0.4127925753344459), np.float64(0.4205340189376448), np.float64(0.5011565122563202), np.float64(0.45776060185262396), np.float64(0.5120051664229581), np.float64(0.5910654533548468), np.float64(0.40989308446900735), np.float64(0.44217657126641197), np.float64(0.5195182931854484), np.float64(0.6451659456391505), np.float64(0.4857703134568306), np.float64(0.4492289245697797), np.float64(0.46623115771216345), np.float64(0.5162772219262227), np.float64(0.42429066722822156), np.float64(0.4378061505156451), np.float64(0.49119254681812957), np.float64(0.5426394886816664), np.float64(0.539274177971466), np.f

In [56]:
mean_bound_brits = []

for i in range(len(lower_bound_brits)):
    mean_bound_brits.append((lower_bound_brits[i]+upper_bound_brits[i])/2)

print(mean_bound_brits)

[np.float64(0.5951056718263642), np.float64(0.6476192480355113), np.float64(0.8009925516357834), np.float64(0.3946049862258218), np.float64(0.5549341598713683), np.float64(0.6508309772554206), np.float64(0.3517318403127996), np.float64(0.6672315173123172), np.float64(0.5910202055794678), np.float64(0.6149646720183661), np.float64(0.6311363441637652), np.float64(0.5149901840326134), np.float64(0.4313759761245102), np.float64(0.4360929093499247), np.float64(0.5075913711274518), np.float64(0.47871150848930927), np.float64(0.5418888102928887), np.float64(0.6321783208600812), np.float64(0.4225271815808874), np.float64(0.48469367501473387), np.float64(0.5333952134082027), np.float64(0.6580834123295086), np.float64(0.5010925320642796), np.float64(0.4683073412101571), np.float64(0.6315212437745643), np.float64(0.5332456705855251), np.float64(0.4484608466950658), np.float64(0.4488183967767265), np.float64(0.5620605615403933), np.float64(0.5584425537485169), np.float64(0.57336486288647), np.floa

## USGAN 

In [57]:
bootstrap_usgan = []

for i in range(len(usgan_ae_mask)):
    bootstrap_usgan.append(toolkits.bootstrap_v2(usgan_ae_mask, i, 9000))

In [58]:
lower_bound_usgan = []
upper_bound_usgan = []

for i in range(len(bootstrap_usgan)):
    lower_bound_usgan.append(np.percentile(bootstrap_usgan[i], 2.5))
    upper_bound_usgan.append(np.percentile(bootstrap_usgan[i], 97.5))

print(lower_bound_usgan)
print(upper_bound_usgan)

[np.float64(0.5180617431308812), np.float64(0.5355525135368893), np.float64(0.4978776867252705), np.float64(0.3399113330447707), np.float64(0.4553267804795127), np.float64(0.44422385624321964), np.float64(0.23143602840608635), np.float64(0.5081546752282472), np.float64(0.5231890879699242), np.float64(0.5205525659975465), np.float64(0.5454300640773584), np.float64(0.42965767026270796), np.float64(0.4313934056332262), np.float64(0.4319716742002413), np.float64(0.4912464449453926), np.float64(0.4442569773619454), np.float64(0.40225228407438257), np.float64(0.5590909562408958), np.float64(0.4383586388858223), np.float64(0.44421974907431494), np.float64(0.518819181475371), np.float64(0.5995971861532075), np.float64(0.49007389907698123), np.float64(0.41939306066414295), np.float64(0.4634819368099337), np.float64(0.4753544528590532), np.float64(0.4356316350115256), np.float64(0.43342628801882704), np.float64(0.4625161187379285), np.float64(0.5053848514888857), np.float64(0.5171478565879942), 

In [59]:
mean_bound_usgan = []

for i in range(len(lower_bound_usgan)):
    mean_bound_usgan.append((lower_bound_usgan[i]+upper_bound_usgan[i])/2)

print(mean_bound_usgan)

[np.float64(0.6130895070798852), np.float64(0.6063555798956477), np.float64(0.5834872527545446), np.float64(0.39289723966939244), np.float64(0.47853289078108624), np.float64(0.503682457373315), np.float64(0.344716174870629), np.float64(0.5426940232074604), np.float64(0.5333482442725737), np.float64(0.5335804512705986), np.float64(0.5552649500536394), np.float64(0.4551286112759626), np.float64(0.44955786027525335), np.float64(0.4502547305852219), np.float64(0.497518653322121), np.float64(0.46516191865899736), np.float64(0.4323477714197456), np.float64(0.5762590986291314), np.float64(0.4567349462499891), np.float64(0.4860363292901145), np.float64(0.5321207804054701), np.float64(0.6119352415986252), np.float64(0.5044708643412193), np.float64(0.43966819614352826), np.float64(0.5724568273234794), np.float64(0.49265031044076807), np.float64(0.4567987946087896), np.float64(0.4445061109931187), np.float64(0.5292381927546224), np.float64(0.515275552887615), np.float64(0.5391178018448055), np.fl

## GPVAE

In [60]:
bootstrap_gpvae = []

for i in range(len(gpvae_ae_mask)):
    bootstrap_gpvae.append(toolkits.bootstrap_v2(gpvae_ae_mask, i, 9000))

In [61]:
lower_bound_gpvae = []
upper_bound_gpvae = []

for i in range(len(bootstrap_gpvae)):
    lower_bound_gpvae.append(np.percentile(bootstrap_gpvae[i], 2.5))
    upper_bound_gpvae.append(np.percentile(bootstrap_gpvae[i], 97.5))

print(lower_bound_gpvae)
print(upper_bound_gpvae)

[np.float64(0.38565849046202133), np.float64(0.49348054988381673), np.float64(0.5781062791136369), np.float64(0.3241797081172884), np.float64(0.4253579843069768), np.float64(0.4447735062158277), np.float64(0.25785283696670824), np.float64(0.5366640653585978), np.float64(0.4646589352819915), np.float64(0.4566623765885701), np.float64(0.4586835697169349), np.float64(0.4476260439269264), np.float64(0.42920524857303943), np.float64(0.4156382676235879), np.float64(0.4568721498843175), np.float64(0.44450144305158057), np.float64(0.44416602069108463), np.float64(0.46051584214177815), np.float64(0.5281043045103231), np.float64(0.43658306758759463), np.float64(0.4453220232243607), np.float64(0.4482099266923245), np.float64(0.43079164646419355), np.float64(0.44129795011457645), np.float64(0.45606356917118523), np.float64(0.4581196008264917), np.float64(0.40115301376431023), np.float64(0.43209740567829885), np.float64(0.42757332144642957), np.float64(0.4475398351033741), np.float64(0.444419285415

In [62]:
mean_bound_gpvae = []

for i in range(len(lower_bound_gpvae)):
    mean_bound_gpvae.append((lower_bound_gpvae[i]+upper_bound_gpvae[i])/2)

print(mean_bound_gpvae)

[np.float64(0.5117635548503001), np.float64(0.5852092902534285), np.float64(0.6582484599754894), np.float64(0.36827920204759046), np.float64(0.4495796070430955), np.float64(0.520471588835504), np.float64(0.34230045835991507), np.float64(0.5652462731159437), np.float64(0.47390860034567406), np.float64(0.46805590924461743), np.float64(0.4674501008617786), np.float64(0.47512559750389544), np.float64(0.44821235362100675), np.float64(0.4316423956491424), np.float64(0.46279550307986295), np.float64(0.4659556049204547), np.float64(0.4759111641041349), np.float64(0.4719059434905739), np.float64(0.5869983361965674), np.float64(0.4775092991003618), np.float64(0.45412993002553026), np.float64(0.45716376193970226), np.float64(0.44050492298161137), np.float64(0.46041347351923007), np.float64(0.47457827966139), np.float64(0.47410659865709115), np.float64(0.42406391821982126), np.float64(0.44328311608322957), np.float64(0.4892082446521361), np.float64(0.4565281936451094), np.float64(0.459267370548778

## MRNN

In [63]:
bootstrap_mrnn = []

for i in range(len(mrnn_ae_mask)):
    bootstrap_mrnn.append(toolkits.bootstrap_v2(mrnn_ae_mask, i, 9000))

In [64]:
lower_bound_mrnn = []
upper_bound_mrnn = []

for i in range(len(bootstrap_mrnn)):
    lower_bound_mrnn.append(np.percentile(bootstrap_mrnn[i], 2.5))
    upper_bound_mrnn.append(np.percentile(bootstrap_mrnn[i], 97.5))

print(lower_bound_mrnn)
print(upper_bound_mrnn)

[np.float64(0.31536944750539697), np.float64(0.19731533687312555), np.float64(0.2687331699664546), np.float64(0.3419372943462037), np.float64(0.33295271288530554), np.float64(0.29791278401905075), np.float64(0.25108602107376965), np.float64(0.3594683565353681), np.float64(0.4168758232820514), np.float64(0.36575891917347086), np.float64(0.2973527003586004), np.float64(0.40846065619335764), np.float64(0.43491447882987894), np.float64(0.40240529566875777), np.float64(0.4035469562784537), np.float64(0.4288978772196218), np.float64(0.325533852348352), np.float64(0.42400533908145266), np.float64(0.1358871984222344), np.float64(0.42449802110693885), np.float64(0.4130991895000895), np.float64(0.42157033883959744), np.float64(0.39649313453524826), np.float64(0.438601538424109), np.float64(0.44058063007052906), np.float64(0.3975516425673767), np.float64(0.38555427001998743), np.float64(0.41310022243353534), np.float64(0.4524346597503807), np.float64(0.4044097270047987), np.float64(0.435323453600

In [65]:
mean_bound_mrnn = []

for i in range(len(lower_bound_mrnn)):
    mean_bound_mrnn.append((lower_bound_mrnn[i]+upper_bound_mrnn[i])/2)

print(mean_bound_mrnn)

[np.float64(0.4459080481210805), np.float64(0.3229539806201789), np.float64(0.39032518942490096), np.float64(0.38408393470236146), np.float64(0.3568149156874806), np.float64(0.3800677037068218), np.float64(0.35934921475709103), np.float64(0.39658586347701363), np.float64(0.42478033861260744), np.float64(0.37638739662488474), np.float64(0.3049988948113582), np.float64(0.43510963044607787), np.float64(0.4538279427422446), np.float64(0.41751962295243533), np.float64(0.40854536223505006), np.float64(0.44949548726330724), np.float64(0.3576939607799824), np.float64(0.43293230114842207), np.float64(0.14073523843326508), np.float64(0.46430538033694435), np.float64(0.4206593222629189), np.float64(0.42950155452156924), np.float64(0.40493194533627963), np.float64(0.4574990124053171), np.float64(0.4590978206588524), np.float64(0.4137319905528455), np.float64(0.40791404835058487), np.float64(0.4241220867498736), np.float64(0.5215610412948001), np.float64(0.41238074588494605), np.float64(0.451177750